In [11]:
# Creat testing csv file
image_dir = '/Users/ranranking/Work/tensorflow_excercise/test_image'
image_list = os.listdir(image_dir)

for img in image_list:
    os.rename(os.path.join(image_dir, img), os.path.join(image_dir, img.replace(' ', '_')))

image_list = np.array([os.path.join(image_dir, img) for img in image_list])

label = np.array([1,1,1,1,0,0,0,0]).astype(str)

image_df = pd.DataFrame([image_list, label]).transpose()

image_df.to_csv('./image.txt', header=None, index=False, sep=' ')

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

In [7]:
NUM_CLASSES = 2

def _parser_function(img_path, label):
    
    # Convert One Hot Label
    one_hot = tf.one_hot(label, NUM_CLASSES)
    
    # Read Image
    img_file = tf.read_file(img_path)
    img_decoded = tf.image.decode_jpeg(img_file, channels=3)
    img_resized = tf.image.resize_images(img_decoded, [256, 256])
    
    return img_resized, one_hot

In [2]:
# Use numpy
data = np.genfromtxt('./image.txt', delimiter=' ', dtype=None)

image_dir = np.array([ele[0] for ele in data])

labels = np.array([ele[1] for ele in data])

dataset = tf.data.Dataset.from_tensor_slices((image_dir, labels))

/Users/ranranking/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  


In [21]:
# Use pandas
data_df = pd.read_csv('./image.txt', sep=' ', header=None)
dataset = tf.data.Dataset.from_tensor_slices((np.array(data_df[0]), np.array(data_df[1])))

In [22]:
dataset = dataset.map(_parser_function).batch(8)

it = dataset.make_one_shot_iterator()

next_element = it.get_next()

summary = tf.summary.image('input', next_element[0], 8)

with tf.Session() as sess:
#     s, labels = sess.run([summary, next_element[1]])
#     print(labels)

    s = sess.run(summary)
    writer = tf.summary.FileWriter('./log/model')
    writer.add_summary(s)
    writer.close()